In [1]:
%pylab tk

Populating the interactive namespace from numpy and matplotlib


In [2]:
from scipy.integrate import odeint
from __future__ import division
import time
import matplotlib.animation as animation
from IPython.display import clear_output, display
from IPython.html.widgets import FloatProgress

:0: FutureWarning: IPython widgets are experimental and may change in the future.


In [198]:
def equilibrate(fa, fb, fp, ka, kb):
    ka, kb = 1.0/ka, 1.0/kb
    a = ka + kb + fa + fb - fp
    b = kb*(fa-fp) + ka*(fb-fp) + ka*kb
    c = -ka*kb*fp
    theta = arccos((-2.0*a*a*a+9.0*a*b-27.0*c)/(2*sqrt((a*a-3.0*b)**3.0)))
    p = -(a/3.0) + (2.0/3.0)*sqrt(a*a-3.0*b)*cos(theta/3.0)
    pa = (fa*(2*sqrt(a*a-3*b)*cos(theta/3.0)-a)) / (3.0*ka+(2*sqrt(a*a-3*b)*cos(theta/3.0)-a))
    pb = (fb*(2*sqrt(a*a-3*b)*cos(theta/3.0)-a)) / (3.0*kb+(2*sqrt(a*a-3*b)*cos(theta/3.0)-a))
    a = fa - pa
    b = fb - pb
    fa / (a+pa)
    return a, b, p, pa, pb

species = 5
# protein, dye, analyte, control
prot_i, dye_i, anal_i, prot_dye_i, prot_anal_i = range(0,species)

def run_column(protein_start=10**-10.0, anal_start=10**-9.0, dye_start=10**-5.5, kanal=10**10.0, kdye=10**8.0, timestep=0.1, \
               verbose=False, dtype=longdouble, stop=2.0):
    
    #width, length, thickness = 8, 50.0, 0.15#units of mm
    #porosity = 0.7
    col_vol = 0.2 #porosity*width*length*thickness / 1000.0 #ml
    if verbose: print 'Column volume is %0.2f ul'%(col_vol*1000)

    flow = 0.01 #ml/min
    flow = flow/60.0 #ml/s
    plates = 100
    plate_vol = col_vol / plates
    plate_flow = flow/plate_vol
    if verbose: print 'Plate volume is %0.2f ul'%(plate_vol*1000)

    vstart, vstop = 0, col_vol*stop
    iterations = int(floor(vstop/(timestep*flow)))
    if verbose: print iterations
    volumes = array(range(0, iterations))*flow*timestep

    if verbose: print 'pf timestep product:',plate_flow*timestep

    col = zeros((plates, species), dtype=dtype)
    col[:, prot_i] = protein_start
    col[:, dye_i] = dye_start
    col[0, anal_i] = anal_start
    

    start = time.time()

    out = zeros((plates, species, iterations))
    for i in range(0, iterations):
        out[:,:,i] = col
        current_volume = i*timestep*flow

        prot = col[:,prot_i]
        dye = col[:,dye_i]
        anal = col[:,anal_i]
        prot_dye = col[:,prot_dye_i]
        prot_anal = col[:,prot_anal_i]

        col[:,anal_i], col[:,dye_i], col[:,prot_i], col[:,prot_anal_i], col[:,prot_dye_i] = \
            equilibrate(anal+prot_anal, dye+prot_dye, prot+prot_dye+prot_anal, kanal, kdye)
        col[isnan(col)] = 0
        
        #flow the column--things that flow are the analyte and the capture (dye) reagent
        for key in [anal_i, dye_i]:
            change = col[:,key].copy()*plate_flow*timestep
            col[:,key] = col[:,key] - change[:]
            col[1:, key] = col[1:, key] + change[:-1]
            
            #continuous input of the analyte at the beginning
            if key == anal_i: col[0, key] = col[0, key] + anal_start*plate_flow*timestep
    out[:,:,i] = col   
        
    if verbose: print "Calculation took %0.2fs"%(time.time()-start)
    return out, iterations, volumes

#testing

In [243]:
protein_start = 10**-7.3
dye_percent = 0.1
anal_start = 10**-7.6
kdye = 1.0/(20*10**-9.0)
kanal = 1.0/(0.5*10**-9.0)
stop = 3.0


out, iterations, volumes = run_column(dye_start=protein_start*dye_percent, anal_start=anal_start, protein_start=protein_start, \
                             kdye = kdye, kanal = kanal, verbose=True, timestep=0.1, \
                            dtype=longdouble, stop=stop)

out_baseline, iterations, volumes = run_column(dye_start=protein_start*dye_percent, anal_start=0.0, protein_start=protein_start, \
                             kdye = kdye, kanal = kanal, verbose=False, timestep=0.1, \
                            dtype=longdouble, stop=stop)

Column volume is 200.00 ul
Plate volume is 2.00 ul
36000
pf timestep product: 0.00833333333333
Calculation took 3.71s


C:\Users\rkwant\AppData\Local\Enthought\Canopy\User\lib\site-packages\ipython-3.0.0-py2.7.egg\IPython\kernel\__main__.py:12: RuntimeWarning: invalid value encountered in true_divide


In [244]:
#animation of the setup
t = 0
dye = out[:,dye_i,t]
prot_dye = out[:,prot_dye_i,t]
prot = out[:,prot_i,t] 
anal = out[:,anal_i,t] 
prot_anal = out[:,prot_anal_i,t]

tprot = prot+prot_dye+prot_anal
tanal = anal+prot_anal
tdye = dye+prot_dye

fig = figure(1, figsize=(10, 5))
clf()

scaling = tprot.max() / anal[0]

c1, c2, c3 =rcParams['axes.color_cycle'][0], rcParams['axes.color_cycle'][1], rcParams['axes.color_cycle'][2]

subplot(211)
dye_line, = plot(dye, '-', label='dye', linewidth=2, color=c1)
prot_dye_line, = plot(prot_dye, '--', label='prot-dye', linewidth=2, color=c1)
anal_line, = plot(anal*scaling, '-', label='anal', color=c2)
prot_anal_line, = plot(prot_anal, '--', label='prot_anal', color=c2)
prot_line, = plot(prot, '--', label='prot', color=c3)
legend(loc='upper right', ncol=3)

ylim((0, tprot.max()*1.2))
grid()
xticks([13, 31], ['analyte_stop', 'protein'])

subplot(212)
together_line, = plot(volumes, zeros(volumes.shape), '-', label='Signal')
together_baseline, = plot(volumes, zeros(volumes.shape), '-', label='Background')
legend()
xlim((volumes.min(), volumes.max()))
ylim((0, out[-1, dye_i-species, :].max()*1.2))
grid()

if False:
    def animate(i):
        i=int(i)
        dye_line.set_ydata(out[:, dye_i, i])
        prot_dye_line.set_ydata(out[:,prot_dye_i,i])
        prot_line.set_ydata(out[:,prot_i,i])
        anal_line.set_ydata(out[:,anal_i,i]*scaling)
        prot_anal_line.set_ydata(out[:,prot_anal_i,i])
        
        toshow = out[-1, dye_i-species, :].copy()
        toshow[i:] = 0
        together_line.set_ydata(toshow)
        
        toshow = out_baseline[-1, dye_i-species, :].copy()
        toshow[i:] = 0
        together_baseline.set_ydata(toshow)

    ani = animation.FuncAnimation(fig, animate, linspace(0, iterations-1, 200),
        interval=10, blit=False, repeat=False)
else: together_line.set_ydata(out[-1, dye_i-species, :])
    
#savefig('animation\\%d-b.pdf'%t)

#heatmaps

In [237]:
#2D heatmap
#analyte concentration
xstart, xstop, xdim = -9, -4.0, 10
#protein concentration
ystart, ystop, ydim = -9, -4.0, 10

kdye, kanalyte = 1.0/(20*10**-9.0), 1.0/(0.5*10**-9.0)
out = zeros((ydim, xdim))

xs, ys = linspace(xstart, xstop, xdim), linspace(ystart, ystop, ydim)

f=FloatProgress(min = 0, max = xdim*ydim)
display(f)
for i,analpow in enumerate(xs):
    for j,dyepow in enumerate(ys):
        f.value=ydim*i+j+1
        rs = []
        for multiplier in [0.0, 1.0]:
            result, iterations, volumes = run_column(anal_start = multiplier*10**analpow, dye_start = 0.1*10**dyepow, \
                                                     protein_start = 10**dyepow, \
                                        kdye=kdye, kanal=kanalyte, timestep=1.0, stop=10.0)
            rs.append(result)
        out[j,i] = rs[1][-1, dye_i-species, :].sum() / rs[0][-1, dye_i-species, :].sum()

C:\Users\rkwant\AppData\Local\Enthought\Canopy\User\lib\site-packages\ipython-3.0.0-py2.7.egg\IPython\kernel\__main__.py:12: RuntimeWarning: invalid value encountered in true_divide


In [238]:
figure(3)
clf()

vmin, vmax = 0, 2

ypad, xpad = 0.5*((ystop-ystart)/(ydim-1)), 0.5*((xstop-xstart)/(xdim-1))
ymin, ymax, xmin, xmax = ystart-ypad, ystop + ypad, xstart-xpad, xstop+xpad

imshow(log10(out), interpolation='none', cmap='rainbow', \
       extent=(xmin, xmax, ymax, ymin), vmin=vmin, vmax=vmax,\
      aspect=(xstop-xstart)/(ystop-ystart))

xticks(xs, ["%0.1f"%i for i in xs], rotation='vertical')
yticks(ys, ["%0.1f"%i for i in ys])


ylabel('[Protein and Dye]')
xlabel('[Analyte]')
xlim((xmin, xmax))
ylim((ymin, ymax))
title('log([P])=%0.2f, log(Kcap)=%0.2f, log(Kanal)=%0.2f'%(pconc, log10(kdye), log10(kanalyte)))

ticks = linspace(vmin, vmax, 10)
cbar = colorbar(ticks=ticks)
cbar.ax.set_yticklabels(['%0.2f'%(10**i) for i in ticks])

tight_layout()
draw()

#savefig('heatmaps\\new-analytical-p_%0.2f-kc_%0.2f-ka_%0.2f.pdf'%(pconc, log10(kdye), log10(kanalyte)))

#heatmap 2

In [226]:
#2D heatmap
#analyte concentration
xstart, xstop, xdim = -9, -4.0, 10
#protein concentration
ystart, ystop, ydim = 0, 1, 10

kdye, kanalyte = 1.0/(20*10**-9.0), 1.0/(100*10**-9.0)
out = zeros((ydim, xdim))
dyepow = -7.0

xs, ys = linspace(xstart, xstop, xdim), linspace(ystart, ystop, ydim)

f=FloatProgress(min = 0, max = xdim*ydim)
display(f)
for i,analpow in enumerate(xs):
    for j,frac in enumerate(ys):
        f.value=ydim*i+j+1
        rs = []
        for multiplier in [0.0, 1.0]:
            result, iterations, volumes = run_column(anal_start = multiplier*10**analpow, dye_start = frac*10**dyepow, \
                                                     protein_start = 10**dyepow, \
                                        kdye=kdye, kanal=kanalyte, timestep=1.0, stop=2.0)
            rs.append(result)
        out[j,i] = rs[1][-1, dye_i-species, :].sum() / rs[0][-1, dye_i-species, :].sum()

C:\Users\rkwant\AppData\Local\Enthought\Canopy\User\lib\site-packages\ipython-3.0.0-py2.7.egg\IPython\kernel\__main__.py:12: RuntimeWarning: invalid value encountered in true_divide
C:\Users\rkwant\AppData\Local\Enthought\Canopy\User\lib\site-packages\ipython-3.0.0-py2.7.egg\IPython\kernel\__main__.py:22: RuntimeWarning: invalid value encountered in double_scalars


In [227]:
figure(4)
clf()

vmin, vmax = 0, 2

ypad, xpad = 0.5*((ystop-ystart)/(ydim-1)), 0.5*((xstop-xstart)/(xdim-1))
ymin, ymax, xmin, xmax = ystart-ypad, ystop + ypad, xstart-xpad, xstop+xpad

imshow(log10(out), interpolation='none', cmap='rainbow', \
       extent=(xmin, xmax, ymax, ymin), vmin=vmin, vmax=vmax,\
      aspect=(xstop-xstart)/(ystop-ystart))

xticks(xs, ["%0.1f"%i for i in xs], rotation='vertical')
yticks(ys, ["%0.1f"%i for i in ys])


ylabel('Fraction occupied')
xlabel('[Analyte]')
xlim((xmin, xmax))
ylim((ymin, ymax))
title('log([P])=%0.2f, log(Kcap)=%0.2f, log(Kanal)=%0.2f'%(pconc, log10(kdye), log10(kanalyte)))

ticks = linspace(vmin, vmax, 10)
cbar = colorbar(ticks=ticks)
cbar.ax.set_yticklabels(['%0.2f'%(10**i) for i in ticks])

tight_layout()
draw()

#savefig('heatmaps\\new-analytical-p_%0.2f-kc_%0.2f-ka_%0.2f.pdf'%(pconc, log10(kdye), log10(kanalyte)))